***GENERATED CODE FOR multiclass PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv("hdfs://34.83.203.43:54310"+eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


def randomforestclassifier(df, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    rf_model = RandomForestClassifier()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    rf_model.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % rf_model.score(X_test, y_test))

    data = {'model': rf_model,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run multiclassHooks.ipynb
try:
	#sourcePreExecutionHook()

	glassdata = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/demo/glass_data.csv', 'filename': 'glass_data.csv', 'token': '', 'delimiter': ','}")
	#sourcePostExecutionHook(glassdata)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run multiclassHooks.ipynb
try:
	#transformationPreExecutionHook()

	manualfe = TransformationMain.run(glassdata,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "column_a", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "21", "mean": "88.52", "stddev": "57.67", "min": "7", "max": "183", "missing": "0"}, "updatedLabel": "column_a"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "column_b", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "21", "mean": "1.52", "stddev": "0.0", "min": "1.51514", "max": "1.52725", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "column_b"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "column_c", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "21", "mean": "13.29", "stddev": "0.48", "min": "12.35", "max": "14.15", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "column_c"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "column_d", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "21", "mean": "3.09", "stddev": "1.08", "min": "0.0", "max": "3.89", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "column_d"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "column_e", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "21", "mean": "1.48", "stddev": "0.58", "min": "0.66", "max": "3.5", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "column_e"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "column_f", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "21", "mean": "72.48", "stddev": "0.91", "min": "69.89", "max": "73.81", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "column_f"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "column_g", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "21", "mean": "0.56", "stddev": "0.32", "min": "0.0", "max": "1.68", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "column_g"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "column_h", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "21", "mean": "8.82", "stddev": "1.28", "min": "5.87", "max": "11.64", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "column_h"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "column_i", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "21", "mean": "0.11", "stddev": "0.48", "min": "0.0", "max": "2.2", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "column_i"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "column_j", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "21", "mean": "0.04", "stddev": "0.07", "min": "0.0", "max": "0.24", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "column_j"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "column_k", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "21", "mean": "2.19", "stddev": "1.5", "min": "1", "max": "6", "missing": "0"}, "updatedLabel": "column_k"}]}))

	#transformationPostExecutionHook(manualfe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run multiclassHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=RandomForestClassifier(manualfe, ["column_a", "column_b", "column_c", "column_d", "column_e", "column_f", "column_g", "column_h", "column_i", "column_j"], "column_k")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
